# Day 26 - 2026/02/06

## 题目列表
- 763. Partition Labels (Medium)
- 45. Jump Game II (Medium)
- 121. Best Time to Buy and Sell Stock (Easy)
- 238. Product of Array Except Self (Medium)
- 189. Rotate Array (Medium)

## ⏱ 预计用时: 3-4小时

In [1]:
from typing import List, Dict, Tuple, Optional
from collections import defaultdict, deque, Counter
import heapq

## 763. Partition Labels (Medium)

给你一个字符串 `s`。我们要把这个字符串划分为尽可能多的片段，同一字母最多出现在一个片段中。

注意，划分结果需要满足：将所有划分结果按顺序连接，得到的字符串仍然是 `s`。

返回一个表示每个字符串片段的长度的列表。

**Example 1:**
- Input: `s = "ababcbacadefegdehijhklij"`
- Output: `[9,7,8]`
- 解释: 划分结果为 `"ababcbaca"`, `"defegde"`, `"hijhklij"`。每个字母最多出现在一个片段中。像 `"ababcbacadefegde"`, `"hijhklij"` 的划分是错误的，因为划分的片段数较少。

**Example 2:**
- Input: `s = "eccbbbbdec"`
- Output: `[10]`

**Constraints:**
- `1 <= s.length <= 500`
- `s` 仅由小写英文字母组成

In [ ]:
class Solution:
    def solve_763(self, s: str) -> List[int]:
        """返回字符串划分后每个片段的长度的列表。"""
        # TODO: 实现你的解法
        res = []
        # 存储最后一次出现的idx
        check = {}
        for i in range(len(s)):
            check[s[i]] = i
        
        # 第二次遍历 - 这是基于你之前的while-loop写的
        # left, find, right = 0, 0, check[s[0]]
        # while find < len(s):
        #     right = max(right, check[s[find]])
        #     if find == right:
        #         res.append(right-left+1)
        #         left = find + 1
        #     find += 1
        
        # 第二次遍历 - 真正的bug在于 -- 你搞错了指针和边界
        # 作为 pointer 的 left，right 和 作为 bound 的 left，right是不一样的
        # 能不能动要想清楚
        # 以及， left<right是binary search的边界，不要混用
        # 以后边界一律 start & end，指针一律left & right
        start, end = 0, 0
        for i in range(len(s)):
            end = max(end, check[s[i]])
            if i >= end:
                res.append(end-start+1)
                start = end + 1
        return res
'''
√ 返回序号list - 指针寻找插入位置
√ 第一遍遍历：每个字母，已出现的，存入hashtable，存入最后发现该字母的idx，如a:[0:8]
如果b完全在a内部，比如b[2:3]，那么b就一定能置入a
如果d完全在a外部，比如[9:11]，那么d就必须得分开
第二遍遍历，left=0, right=a[1], 从idx较早的开始，if ==a, 跳过 if b and b[1]<a[1] 跳过, 遇到d and d[1]>a[1]，right = d[1], 直到left>right
---> 

你的直觉是对的："如果 b 完全在 a 内部，b 就能跟 a 在同一个片段"

但问题在于实现时：
你用了两层判断逻辑（if left == right + if check[s[left]] > check[s[right]]），导致逻辑分支太多，容易遗漏边界
你把"遍历"和"边界扩展"混在一起了——left 既要遍历每个字符，又要判断是否到达切分点
'''

def test_solve_763():
    s = Solution()
    # 示例用例
    assert s.solve_763("ababcbacadefegdehijhklij") == [9,7,8]
    assert s.solve_763("eccbbbbdec") == [10]
    
    # 单字符
    assert s.solve_763("a") == [1]
    
    # 所有字符相同
    assert s.solve_763("aaaa") == [4]
    
    # 每个字符都不同
    assert s.solve_763("abcde") == [1,1,1,1,1]
    print("All test cases for 763 passed!")


test_solve_763()

All test cases for 763 passed!


## 45. Jump Game II (Medium)

给你一个非负整数数组 `nums`，你最初位于数组的第一个位置。

数组中的每个元素代表你在该位置可以跳跃的最大长度。

你的目标是使用**最少的跳跃次数**到达数组的最后一个位置。

假设你总是可以到达数组的最后一个位置。

**Example 1:**
- Input: `nums = [2,3,1,1,4]`
- Output: `2`
- 解释: 跳到最后一个位置的最小跳跃数是 `2`。从下标为 `0` 跳到下标为 `1` 的位置，跳 `1` 步，然后跳 `3` 步到达数组的最后一个位置。

**Example 2:**
- Input: `nums = [2,3,0,1,4]`
- Output: `2`

**Constraints:**
- `1 <= nums.length <= 10^4`
- `0 <= nums[i] <= 1000`
- 题目保证可以到达 `nums[n-1]`

In [ ]:
class Solution:
    def solve_45_dp(self, nums: List[int]) -> int:
        """返回到达数组最后一个位置的最少跳跃次数。"""
        # TODO: 实现你的解法
        # DP做法
        dp = [float('inf')] * len(nums)
        dp[0] = 0
        for i in range(1, len(nums)):
            for j in range(i):
                if j + nums[j] >= i:
                    dp[i] = min(dp[i], dp[j]+1)
        return dp[-1]

    def solve_45(self, nums: List[int]) -> int:    
        # greedy 做法
        # 每次跳的时候，可去到的区间是 [i:i+nums[i]]，那么这里面的i，你都是可以看见的，可以取这里面的最优        
        jumps = 0
        current_end = 0
        farthest = 0
        # 不遍历最后一个
        # 因为题目问的是"到达最后一个位置的最少跳跃次数"，而不是"从最后一个位置再跳一次"。
        for i in range(len(nums) - 1):
            farthest = max(farthest, i+nums[i])
            if i == current_end:
                current_end = farthest
                jumps += 1
        return jumps
'''
保证可以抵达，寻找最小次数，那说明满足条件的不止一条
对每个位置来说，最重要的是知道上一步从哪里跳过来的
如果像jump I那样往前，一直取最大，很容易出现局部最优 != 全局最优的情况
这道题的难点就是，未来的值可能影响当下取值
暴力解法，dp，dp[i]表示到达此地的最小次数
'''

def test_solve_45():
    s = Solution()
    # 示例用例
    # print(s.solve_45([2,3,1,1,4]))
    assert s.solve_45([2,3,1,1,4]) == 2
    assert s.solve_45([2,3,0,1,4]) == 2
    
    # 单元素
    assert s.solve_45([1]) == 0
    
    # 一步到达
    assert s.solve_45([2,1]) == 1
    
    # 需要多步
    assert s.solve_45([1,1,1,1]) == 3
    print("All test cases for 45 passed!")


test_solve_45()

All test cases for 45 passed!


## 121. Best Time to Buy and Sell Stock (Easy)

给定一个数组 `prices`，它的第 `i` 个元素 `prices[i]` 表示一支给定股票第 `i` 天的价格。

你只能选择**某一天**买入这只股票，并选择在**未来的某一个不同的日子**卖出该股票。设计一个算法来计算你所能获取的最大利润。

返回你可以从这笔交易中获取的最大利润。如果你不能获取任何利润，返回 `0`。

**Example 1:**
- Input: `prices = [7,1,5,3,6,4]`
- Output: `5`
- 解释: 在第 2 天（股票价格 = 1）的时候买入，在第 5 天（股票价格 = 6）的时候卖出，最大利润 = 6-1 = 5。注意利润不能是 7-1 = 6, 因为卖出价格需要大于买入价格；同时，你不能在买入前卖出股票。

**Example 2:**
- Input: `prices = [7,6,4,3,1]`
- Output: `0`
- 解释: 在这种情况下, 交易无法完成, 所以最大利润为 `0`。

**Constraints:**
- `1 <= prices.length <= 10^5`
- `0 <= prices[i] <= 10^4`

In [ ]:
class Solution:
    def solve_121(self, prices: List[int]) -> int:
        """返回买卖股票能获取的最大利润。"""
        # TODO: 实现你的解法
        # 只要维护"当前遇到的最低价"，当前利润一定是最优的。
        buy, profit = float('inf'), 0
        for p in prices:
            if p <= buy:
                buy = p
            else:
                profit = max(profit, p-buy)
        return profit

'''
要得知利润最大，需要找到prices[i]和prices[j]，满足i<j 且 max(prices[j]-prices[i])
如果用dp，那可以在O(n^2)的时间下算出最大值
遍历整个数组 p, 设定买入价buy，如果 p < buy，可以换天买入
如果 p > buy，就可以考虑卖出
'''

def test_solve_121():
    s = Solution()
    # 示例用例
    assert s.solve_121([7,1,5,3,6,4]) == 5
    assert s.solve_121([7,6,4,3,1]) == 0
    
    # 单元素
    assert s.solve_121([1]) == 0
    
    # 价格一直上涨
    assert s.solve_121([1,2,3,4,5]) == 4
    
    # 价格一直下跌
    assert s.solve_121([5,4,3,2,1]) == 0
    print("All test cases for 121 passed!")


test_solve_121()

All test cases for 121 passed!


## 238. Product of Array Except Self (Medium)

给你一个整数数组 `nums`，返回数组 `answer`，其中 `answer[i]` 等于 `nums` 中除 `nums[i]` 之外其余各元素的乘积。

题目数据**保证**数组 `nums` 之中任意元素的全部前缀元素和后缀的乘积都在**32 位整数**范围内。

请**不要使用除法**，且在 `O(n)` 时间复杂度内完成此题。

**Example 1:**
- Input: `nums = [1,2,3,4]`
- Output: `[24,12,8,6]`

**Example 2:**
- Input: `nums = [-1,1,0,-3,3]`
- Output: `[0,0,9,0,0]`

**Constraints:**
- `2 <= nums.length <= 10^5`
- `-30 <= nums[i] <= 30`
- 保证数组 `nums` 之中任意元素的全部前缀元素和后缀的乘积都在**32 位整数**范围内

In [ ]:
class Solution:
    def solve_238_self_done(self, nums: List[int]) -> List[int]:
        """返回除自身外其他元素的乘积数组。"""
        # TODO: 实现你的解法
        prefix, suffix = [1]*len(nums), [1]*len(nums)
        for i in range(1, len(nums)):
            prefix[i] = prefix[i-1] * nums[i-1]
        for i in range(len(nums)-2, -1, -1):
            suffix[i] = suffix[i+1] * nums[i+1]
        res = []
        for i in range(len(nums)):
            res.append(prefix[i] * suffix[i])
        return res

        # 空间优化版本
    def solve_238(self, nums: List[int]) -> List[int]:
        res = [1] * len(nums)
        # prefix
        for i in range(1, len(nums)):
            res[i] = res[i-1] * nums[i-1]
            
        # suffix - 滚动后缀积
        r = 1
        for i in range(len(nums)-1, -1, -1):
            res[i] *= r
            r *= nums[i]
        return res
'''
On内解决，返回数组为每个数除了自己以外的乘积，不能用除法
那对于idx=i的数字，其要填入的内容为 nums[:i] * nums[i+1:]
维护一个前缀积，维护一个后缀积，合起来就好了
时间为 O 3*n， 空间位 0 3*n，合起来时空都是 On
'''

def test_solve_238():
    s = Solution()
    # 示例用例
    assert s.solve_238([1,2,3,4]) == [24,12,8,6]
    assert s.solve_238([-1,1,0,-3,3]) == [0,0,9,0,0]
    
    # 两个元素
    assert s.solve_238([1,2]) == [2,1]
    
    # 包含负数
    assert s.solve_238([-1,-2,-3]) == [6,3,2]
    
    # 包含0
    assert s.solve_238([0,0]) == [0,0]
    print("All test cases for 238 passed!")


test_solve_238()

All test cases for 238 passed!


## 189. Rotate Array (Medium)

给定一个整数数组 `nums`，将数组中的元素向右轮转 `k` 个位置，其中 `k` 是非负数。

**Example 1:**
- Input: `nums = [1,2,3,4,5,6,7]`, `k = 3`
- Output: `[5,6,7,1,2,3,4]`
- 解释: 向右轮转 3 步: `[1,2,3,4,5,6,7]` => `[7,1,2,3,4,5,6]` => `[6,7,1,2,3,4,5]` => `[5,6,7,1,2,3,4]`

**Example 2:**
- Input: `nums = [-1,-100,3,99]`, `k = 2`
- Output: `[3,99,-1,-100]`
- 解释: 向右轮转 2 步: `[-1,-100,3,99]` => `[99,-1,-100,3]` => `[3,99,-1,-100]`

**Constraints:**
- `1 <= nums.length <= 10^5`
- `-2^31 <= nums[i] <= 2^31 - 1`
- `0 <= k <= 10^5`

**进阶：**
- 尽可能想出更多的解决方案，至少有**三种**不同的方法可以解决这个问题。
- 你可以使用空间复杂度为 `O(1)` 的**原地**算法解决这个问题吗？

In [43]:
class Solution:
    def solve_189(self, nums: List[int], k: int) -> None:
        """将数组中的元素向右轮转 k 个位置（原地修改）。"""
        # TODO: 实现你的解法（原地修改 nums）
        # O space = 1
        # !!! k要先取模，k有可能大于nums的 !!!
        '''
        取模，开闭区间取值，切片即副本非原地。———这里有三个坑。
        '''
        def reverse(left, right):
            while left <= right:
                nums[left], nums[right] = nums[right], nums[left]
                left += 1
                right -= 1
        k %= len(nums)
        reverse(0, len(nums)-k-1)
        reverse(len(nums)-k, len(nums)-1)
        reverse(0, len(nums)-1)

    def solve_189_On(self, nums: List[int], k: int) -> None:
        """将数组中的元素向右轮转 k 个位置（原地修改）。"""
        # TODO: 常规解法
        return nums[len(nums)-k:] + nums[:len(nums)-k]
    
'''
原地方法只有一个。
旋转k个位置，其实nums里的每个元素都进行了改变，如果要原地挪的话，得每个都动

小坑：nums[:len(nums)-k] 返回的是引用，不是原地！！！
这道题要原地处理必须是指针控制
还有就是k取模，每次操作idx都要考虑是否存在idx out of range的可能性
'''

def test_solve_189():
    s = Solution()
    # 示例用例
    nums1 = [1,2,3,4,5,6,7]
    s.solve_189(nums1, 3)
    assert nums1 == [5,6,7,1,2,3,4]
    nums2 = [-1,-100,3,99]
    s.solve_189(nums2, 2)
    assert nums2 == [3,99,-1,-100]
    
    # k 等于数组长度
    nums3 = [1,2,3]
    s.solve_189(nums3, 3)
    assert nums3 == [1,2,3]
    
    # k 大于数组长度
    nums4 = [1,2]
    s.solve_189(nums4, 3)
    assert nums4 == [2,1]
    
    # 单元素
    nums5 = [1]
    s.solve_189(nums5, 1)
    assert nums5 == [1]
    print("All test cases for 189 passed!")

test_solve_189()

All test cases for 189 passed!


## 📝 Notes

(在这里记录你的学习笔记)